In [2]:
## Import Libraries

import pandas as pd
import numpy as np
import string
import re
import nltk

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.manifold import TSNE    

from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from imblearn.over_sampling import SMOTE

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/hannahpetry/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [3]:
# load data
df = pd.read_csv('../data/raw/mtsamples.csv')
df.transcription=df.transcription.astype(str)

In [4]:
# remove rows with missing values
def clean_df(data):
    df = data.dropna().drop_duplicates() 
    # drop rows where medical specialty appears less than 100 times
    df = df[df.groupby("medical_specialty")["medical_specialty"].transform('size') > 100]
    print("Number of rows after removing medical specialties with less than 100 samples:", len(df.index))
    # remove unnecessary columns, only keep transcriptions and medical_specialty columns
    return df[['transcription', 'medical_specialty']]


# remove punctuation and lowercase and lemmatizer
stop = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def remove_punct_lower(data):
    '''remove punctuation and lowercase'''
    data["transcription_c"] = data["transcription"].apply(lambda x: x.lower().translate(str.maketrans('','', string.punctuation)))
    return data

def lemmatize_words(data):
    '''lemmatize words, remove stopwords'''
    data["transcription_c"] = data["transcription_c"].apply(lambda x: [lemmatizer.lemmatize(x) for x in word_tokenize(x) if x not in (stop)])
    return data
 


#apply on dataset
df_m = clean_df(df)
df_test = remove_punct_lower(df_m)
df_test = lemmatize_words(df_test)


df_test.head(2)

Number of rows after removing medical specialties with less than 100 samples: 2976


,transcription,medical_specialty,transcription_c
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary,"[2d, mmode, 1, left, atrial, enlargement, left..."
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary,"[1, left, ventricular, cavity, size, wall, thi..."


In [5]:
# NLP with Spacy
import spacy
import en_ner_bionlp13cg_md
nlp = en_ner_bionlp13cg_md.load()
def medical_entities( text):
    entities = []
    doc = nlp(text)
    for ent in doc.ents:
        entities.append(ent.text)
    return set(entities)#' ,'.join(entities)


df_test['transcription_c'] = [','.join(map(str, l)) for l in df_test['transcription_c']]
df_test['transcription_f'] = df_test['transcription_c'].apply(medical_entities)
df_test.head()

,transcription,medical_specialty,transcription_c,transcription_f
3,"2-D M-MODE: , ,1. Left atrial enlargement wit...",Cardiovascular / Pulmonary,"2d,mmode,1,left,atrial,enlargement,left,atrial...","{mitral, valve, ventricular, pulmonary, left}"
4,1. The left ventricular cavity size and wall ...,Cardiovascular / Pulmonary,"1,left,ventricular,cavity,size,wall,thickness,...","{lipomatous, mitral, leaflet, wall, root, inte..."
7,"2-D ECHOCARDIOGRAM,Multiple views of the heart...",Cardiovascular / Pulmonary,"2d,echocardiogrammultiple,view,heart,great,ves...","{aorta, mitral, interatrial, heart, vessel, ve..."
9,"DESCRIPTION:,1. Normal cardiac chambers size....",Cardiovascular / Pulmonary,"description1,normal,cardiac,chamber,size2,norm...","{mitral, valve, ventricular, cardiac, left}"
11,"2-D STUDY,1. Mild aortic stenosis, widely calc...",Cardiovascular / Pulmonary,"2d,study1,mild,aortic,stenosis,widely,calcifie...","{mitral, heart, ventricle, ventricular, left}"


In [6]:
# retrieve labels as function
def get_labels(data):
    return data['medical_specialty'].tolist()

df_test_label = get_labels(df_test)

df_test_X = df_test['transcription_f'].astype(str)

In [7]:
# split data into train and test set as function
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(df_test_X, df_test_label)

In [8]:
# vectorize transcription to receive sparse matrix
def vectorize(data):
    vectorizer = CountVectorizer().fit_transform(data)
    return vectorizer
df_test_X_vectorized = vectorize(df_test_X)
df_test_X_vectorized

<2976x2702 sparse matrix of type '<class 'numpy.int64'>'
	with 50796 stored elements in Compressed Sparse Row format>

In [9]:
# balance out dataset with SMOTE, creates synthetic samples of the minority classes as function
def smote(X, y):
    sm = SMOTE(random_state=42)
    X_res, y_res = sm.fit_resample(X, y)
    return X_res, y_res


X_res, y_res = smote(df_test_X_vectorized, df_test_label)
X_res, y_res

(<11253x2702 sparse matrix of type '<class 'numpy.int64'>'
 	with 205251 stored elements in Compressed Sparse Row format>,
 [' Cardiovascular / Pulmonary',
  ' Cardiovascular / Pulmonary',
  ' Cardiovascular / Pulmonary',
  ' Cardiovascular / Pulmonary',
  ' Cardiovascular / Pulmonary',
  ' Cardiovascular / Pulmonary',
  ' Urology',
  ' General Medicine',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology',
  ' Urology'

In [10]:
# pca for dimensionality reduction
def pca(data):
    pca = PCA(n_components=0.95)
    X_reduced = pca.fit_transform(data.toarray())
    return X_reduced

df_test_X_pca = pca(df_test_X_vectorized)

In [27]:
def build_preprocessing_pipeline(data):
    return Pipeline([
        ('vectorize', FunctionTransformer(vectorize, validate=False))
    ])

X_train_prepared = build_preprocessing_pipeline(X_train)

def build_preprocessing_pipeline_2():
    return Pipeline([
        ('vectorize', CountVectorizer())
    ])
X_train_prepared_2 = build_preprocessing_pipeline_2().fit_transform(X_train)
X_train_prepared_2

<2380x2574 sparse matrix of type '<class 'numpy.int64'>'
	with 40649 stored elements in Compressed Sparse Row format>

In [12]:
def lr_model(data):
    lr = LogisticRegression(random_state=42, multi_class='multinomial')
    return lr

In [25]:
def build_pipeline(x_train, y_train):
    preprocessing_pipeline = build_preprocessing_pipeline()
    return Pipeline([
        ('preprocessing', preprocessing_pipeline),
        ('smote', FunctionTransformer(smote, validate=False)),
        ('pca', FunctionTransformer(pca, validate=False)),
        ('classifier', FunctionTransformer(lr_model, validate=False)) 
    ])

def build_pipeline_2():
    preprocessing_pipeline = build_preprocessing_pipeline_2()
    return Pipeline([
        ('preprocessing', preprocessing_pipeline),
        ('smote', SMOTE(random_state=42)),
        ('pca', PCA(n_components=0.95)),
        ('classifier', LogisticRegression(random_state=42, multi_class='multinomial')) 
])

In [29]:
param_grid = [
    { 'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],
      'classifier': [LogisticRegression(multi_class='multinomial', random_state=42)],
      'classifier__solver': ['saga', 'lbfgs', 'liblinear'],
      'classifier__penalty': ['none', 'l1', 'l2', 'elasticnet'],
    }
]

pipe = build_pipeline_2().fit_transform(X_train)
# pipe.fit(X_train, y_train)

#search = GridSearchCV(pipe, param_grid, cv=5)
#search.fit(X_train, y_train)
#search.best_params_

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'SMOTE(random_state=42)' (type <class 'imblearn.over_sampling._smote.base.SMOTE'>) doesn't